In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.torch_imports import *
from fastai.dataset import *
from fastai.learner import *
from fastai.structured import *
from fastai.column_data import *

In [3]:
PATH = "data/"

In [ ]:
df_raw = pd.read_csv("data/sales_train.csv", low_memory=False,
                     parse_dates=["date"])

In [ ]:
df_raw.head()

In [ ]:
add_datepart(df_raw, 'saledate')

m = RandomForestRegressor(n_jobs=-1)
m.fit(df_raw.drop('item_cnt_day', axis=1), df_raw.item_cnt_day)